In [2]:
!gdown https://drive.google.com/uc?id=107Qo3Fs_pdzByrOMpet4w8G0uNle2ooN

import zipfile
with zipfile.ZipFile("complexnn.zip","r") as zip_ref:
    zip_ref.extractall()

Downloading...
From: https://drive.google.com/uc?id=107Qo3Fs_pdzByrOMpet4w8G0uNle2ooN
To: /content/complexnn.zip
100% 12.6k/12.6k [00:00<00:00, 20.3MB/s]


In [3]:
import pandas as pd
import numpy as np
import os

from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import seaborn as sns
import cv2
from sklearn.model_selection import train_test_split
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [9]:
!gdown https://drive.google.com/uc?id=1tFoQJrwtMYCBMG0WH6unek9dc2sjfQZ4

import zipfile
with zipfile.ZipFile("breast-histopathology-images.zip","r") as zip_ref:
    zip_ref.extractall("breast-histopathology-images")

Downloading...
From: https://drive.google.com/uc?id=1tFoQJrwtMYCBMG0WH6unek9dc2sjfQZ4
To: /content/breast-histopathology-images.zip
3.33GB [00:53, 62.4MB/s]


In [10]:
imagePatches = glob('breast-histopathology-images/**/*.png', recursive=True)
for filename in imagePatches[0:10]:
    print(filename)

breast-histopathology-images/12751/1/12751_idx5_x2001_y1201_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1101_y2301_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1551_y2001_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1451_y2101_class1.png
breast-histopathology-images/12751/1/12751_idx5_x2351_y2051_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1951_y1401_class1.png
breast-histopathology-images/12751/1/12751_idx5_x2451_y2351_class1.png
breast-histopathology-images/12751/1/12751_idx5_x501_y2101_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1251_y2351_class1.png
breast-histopathology-images/12751/1/12751_idx5_x1701_y2401_class1.png


In [11]:
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)
print("IDC(-)\n\n",classZero[0:5],'\n')
print("IDC(+)\n\n",classOne[0:5])

IDC(-)

 ['breast-histopathology-images/12751/0/12751_idx5_x1301_y451_class0.png', 'breast-histopathology-images/12751/0/12751_idx5_x1551_y1451_class0.png', 'breast-histopathology-images/12751/0/12751_idx5_x1201_y1501_class0.png', 'breast-histopathology-images/12751/0/12751_idx5_x1901_y1001_class0.png', 'breast-histopathology-images/12751/0/12751_idx5_x801_y1501_class0.png'] 

IDC(+)

 ['breast-histopathology-images/12751/1/12751_idx5_x2001_y1201_class1.png', 'breast-histopathology-images/12751/1/12751_idx5_x1101_y2301_class1.png', 'breast-histopathology-images/12751/1/12751_idx5_x1551_y2001_class1.png', 'breast-histopathology-images/12751/1/12751_idx5_x1451_y2101_class1.png', 'breast-histopathology-images/12751/1/12751_idx5_x2351_y2051_class1.png']


In [0]:
X=[]
y=[]

for u in classZero:
  X.append(u)
  y.append(0)

for u in classOne:
  X.append(u)
  y.append(1)



In [0]:
df = pd.DataFrame()
df["images"]=X
df["labels"]=y
df=df.sample(frac=1)

In [0]:
def RgbToQuaternion(img):
 b,g,r = cv2.split(img)

 M=np.zeros((50,50,4))
 M[:,:,1]=r
 M[:,:,2]=g
 M[:,:,3]=b
 return(np.array(M))


In [0]:
def batch_generator(df,batch_size,steps):
 idx=1
 while True: 
  yield load_data(df,idx-1,batch_size)## Yields data
  if idx < steps:
    idx+=1
  else:
    idx=1

In [0]:
def load_data(df,idx,batch_size):
 start=idx*batch_size
 end=(idx+1)*batch_size
 x = []
 y = [] 
 for  row  in df.values[start:end,:]:
  img=row[0]

  full_size_image = cv2.imread(img)
  X=cv2.resize(full_size_image, (50,50), interpolation=cv2.INTER_CUBIC)

  image = RgbToQuaternion(X) /255.
  x.append(image) 
  y.append(row[-1] )

 return (np.array(x).reshape(-1,50,50,4), np.array(y).reshape(-1))

In [0]:

df_train, df_test = train_test_split(df, test_size=0.3)

In [0]:
train_batch_generator = batch_generator(df_train, 64,len(df_train)//64)
test_batch_generator = batch_generator(df_test, 64,len(df_train)//64)

In [4]:
from   complexnn                             import *
import keras
from   keras.layers                          import *
from   keras.models                          import Model
import keras.backend                         as     K
import numpy                                 as     np

Using TensorFlow backend.


In [0]:


from keras.optimizers import adam
opt = adam(lr = 0.001,decay=1e-6)

In [0]:
input_seq = Input((50, 50, 4))
O    =QuaternionConv2D(64, (3,3),activation='relu',padding='same')(input_seq)  
O    = MaxPooling2D((2,2), padding='same')(O)

O    =QuaternionConv2D(128, (3,3),activation='relu',padding='same')(input_seq)  
O    = MaxPooling2D((2,2), padding='same')(O)

O    =QuaternionConv2D(256, (3,3),activation='relu',padding='same')(input_seq)  
O    = MaxPooling2D((2,2), padding='same')(O)

O    = Flatten()(O)

O  = QuaternionDense(256, activation='relu')(O)
            
O = Dense(1, activation='sigmoid')(O)
model=Model(input_seq, O)
            
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
            
            

In [0]:
model.fit_generator(batch_generator(df_train, 128,len(df_train)//128),\
                    steps_per_epoch=len(df_train)//128,epochs=10,\
                    verbose=1,validation_data=batch_generator(df_test, 64,len(df_test)//64),validation_steps=len(df_test)//64)

Epoch 1/10
 630/3035 [=====>........................] - ETA: 8:05 - loss: 8.8304 - accuracy: 0.8070